In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% ! important;}
div.cell.code_cell.rendered{width:100%}
div.input_prompt{padding:0px}
div.CodeMirror {font-family:Consolas ; font-size:12pt;}
div.text_cell_render.rendered_html {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold}
div.input {font-family:Consolas ; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">웹데이터 수집 주가지수</font></b>
## 주가 웹크롤링

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import date

In [3]:
# krx 전체지수 시세 창 open
driver = webdriver.Chrome()
driver.get("https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101")
time.sleep(2) # 데이터 로딩시간의 문제로 추가

In [4]:
# 지수명, 종가, 거래량을 각각의 리스트에 저장
krx_name_list =[]
closing_price_list =[]
trade_value_list = []
krx_list =[]

krx_name = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-LEFT[data-name="IDX_NM"]')
closing_price = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="CLSPRC_IDX"]')
trade_value_list = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="ACC_TRDVOL"]')

krx_name_list = [items.text for items in krx_name if items.text != ""]
closing_price_list = [items.text for items in closing_price if items.text != ""]
trade_value_list = [items.text for items in trade_value_list if items.text != ""]

In [5]:
# 각각의 리스트의 값들을 2차원 배열로 순서에 맞게 저장
for idx, item, in enumerate(krx_name_list):
    krx_list.append([krx_name_list[idx], closing_price_list[idx], trade_value_list[idx]])
    
krx_list

[['코리아 밸류업 지수', '1,722.17', '78,098'],
 ['KRX TMI', '2,522.55', '1,093,697'],
 ['KRX 300', '2,643.47', '168,202'],
 ['KRX 중대형 TMI', '2,537.47', '358,122'],
 ['KRX 중형 TMI', '1,688.93', '189,920'],
 ['KRX 소형 TMI', '1,537.56', '490,010'],
 ['KRX 초소형 TMI', '5,472.91', '245,565'],
 ['KTOP 30', '11,324.16', '53,265'],
 ['KRX 100', '9,132.91', '96,401'],
 ['KRX 자동차', '2,496.37', '12,349'],
 ['KRX 반도체', '6,077.62', '37,924'],
 ['KRX 헬스케어', '4,775.53', '19,621'],
 ['KRX 은행', '1,307.17', '6,836'],
 ['KRX 에너지화학', '2,611.80', '4,747'],
 ['KRX 철강', '2,543.14', '3,945'],
 ['KRX 방송통신', '795.90', '1,199'],
 ['KRX 건설', '832.23', '39,530'],
 ['KRX 증권', '1,570.67', '22,523'],
 ['KRX 기계장비', '1,508.60', '30,537'],
 ['KRX 보험', '2,703.82', '6,406']]

In [161]:
# 필요한 키워드만 헤당 날짜와 함께 저장
keywords = ['건설', '헬스케어', '자동차'] 
fdate = driver.find_element(By.CSS_SELECTOR, 'input[name="trdDd"]')
date = fdate.get_attribute("value")

                                                                                                                 # 주말 혹은 공휴일의 데이터는 '-', 저장을 막기위해
filtered = [ [date, name, price, trade] for name, price, trade in krx_list if any(kw in name for kw in keywords) if (price != '-') or (trade != '-')] 
print(filtered)

[['20251219', 'KRX 자동차', '2,515.84', '12,711'], ['20251219', 'KRX 헬스케어', '4,861.55', '29,995'], ['20251219', 'KRX 건설', '820.60', '35,260']]


## 날짜변경

In [9]:
from datetime import datetime, timedelta

# 오늘 날짜
today = datetime.today()

# 지난 달 같은 날짜
last_month = today.replace(month=today.month - 1)

# 하루씩 감소하면서 출력
current = today
while current >= last_month:
    print(current.strftime("%Y-%m-%d"))
    current -= timedelta(days=1)

2025-12-23
2025-12-22
2025-12-21
2025-12-20
2025-12-19
2025-12-18
2025-12-17
2025-12-16
2025-12-15
2025-12-14
2025-12-13
2025-12-12
2025-12-11
2025-12-10
2025-12-09
2025-12-08
2025-12-07
2025-12-06
2025-12-05
2025-12-04
2025-12-03
2025-12-02
2025-12-01
2025-11-30
2025-11-29
2025-11-28
2025-11-27
2025-11-26
2025-11-25
2025-11-24
2025-11-23


In [154]:
# 날짜 변경, 조회일자 형식이 yyyymmdd형식이라 조회일자 입력창에 들어가는 변수엔 .strftime("%Y%m%d")을 추가해야 함
new_date = "20251213" # 테스트용
date_input = driver.find_element(By.ID, "trdDd")
driver.execute_script("arguments[0].value = arguments[1];", date_input, new_date)

# 변경된 날짜로 조회
search_btn = driver.find_element(By.ID, "jsSearchButton") 
search_btn.click()

## 실행코드

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta
import pandas as pd

# krx 전체지수 시세 창 open
driver = webdriver.Chrome()
driver.get("https://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101")
time.sleep(4) # 데이터 로딩시간의 문제로 추가 3초는 가끔씩 로딩시간에 걸림

In [10]:
input_date = int(input('몇일 전 까지 크롤링할까요? '))
now_day = today
chg_date = now_day

krx_name_list =[]
closing_price_list =[]
trade_value_list = []
krx_list =[]
result_list =[]

for date_idx in range(input_date):
    
    krx_name = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-LEFT[data-name="IDX_NM"]')
    closing_price = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="CLSPRC_IDX"]')
    trade_value_list = driver.find_elements(By.CSS_SELECTOR, 'td.CI-GRID-ALIGN-RIGHT[data-name="ACC_TRDVOL"]')
    
    krx_name_list = [items.text for items in krx_name if items.text != ""]
    closing_price_list = [items.text.replace(",","") for items in closing_price if items.text != ""]
    trade_value_list = [items.text.replace(",","") for items in trade_value_list if items.text != ""]
    date_maker = chg_date.strftime("%Y%m%d")
    
    for idx, item, in enumerate(krx_name_list):
        krx_list.append([date_maker, krx_name_list[idx], closing_price_list[idx], trade_value_list[idx]])
    keywords = ['건설', '헬스케어', '자동차'] 
    fdate = driver.find_element(By.CSS_SELECTOR, 'input[name="trdDd"]')
                                                                                                     
    result_list = [ [date, name, price, trade] for date, name, price, trade in krx_list if any(kw in name for kw in keywords) if (price != '-') or (trade != '-')] 

    chg_date -= timedelta(days=1)
    date_input = driver.find_element(By.ID, "trdDd")
    driver.execute_script("arguments[0].value = arguments[1];", date_input, chg_date.strftime("%Y%m%d"))

    search_btn = driver.find_element(By.ID, "jsSearchButton") 
    search_btn.click()
    
    time.sleep(4)
    
# 오늘날짜로 변경
chg_date = now_day
date_input = driver.find_element(By.ID, "trdDd")
driver.execute_script("arguments[0].value = arguments[1];", date_input, chg_date.strftime("%Y%m%d"))

search_btn = driver.find_element(By.ID, "jsSearchButton") 
search_btn.click()

몇일 전 까지 크롤링할까요? 28


In [11]:
# 리스트 확인
result_list[::3] #하루당 3행이므로 3행씩 출력하여 확인

[['20251223', 'KRX 자동차', '2496.37', '12349'],
 ['20251222', 'KRX 자동차', '2523.31', '13166'],
 ['20251219', 'KRX 자동차', '2515.84', '12806'],
 ['20251218', 'KRX 자동차', '2459.08', '9597'],
 ['20251217', 'KRX 자동차', '2509.42', '20114'],
 ['20251216', 'KRX 자동차', '2485.66', '14758'],
 ['20251215', 'KRX 자동차', '2532.83', '8978'],
 ['20251212', 'KRX 자동차', '2579.65', '10633'],
 ['20251211', 'KRX 자동차', '2516.66', '16310'],
 ['20251210', 'KRX 자동차', '2547.48', '10938'],
 ['20251209', 'KRX 자동차', '2559.72', '12216'],
 ['20251208', 'KRX 자동차', '2604.74', '14134'],
 ['20251205', 'KRX 자동차', '2607.52', '22724'],
 ['20251204', 'KRX 자동차', '2486.75', '21996'],
 ['20251203', 'KRX 자동차', '2415.51', '14140'],
 ['20251202', 'KRX 자동차', '2376.20', '24411'],
 ['20251201', 'KRX 자동차', '2294.57', '11789'],
 ['20251128', 'KRX 자동차', '2331.62', '17723'],
 ['20251127', 'KRX 자동차', '2296.52', '14199'],
 ['20251126', 'KRX 자동차', '2262.07', '11557']]

In [12]:
# 리스트를 데이터프레임으로
import pandas as pd
df = pd.DataFrame(result_list, columns=['날짜', '종목명', '종가(백만원)', '거래량(천주)'])
df

,날짜,종목명,종가(백만원),거래량(천주)
0,20251223,KRX 자동차,2496.37,12349
1,20251223,KRX 헬스케어,4775.53,19621
2,20251223,KRX 건설,832.23,39530
3,20251222,KRX 자동차,2523.31,13166
4,20251222,KRX 헬스케어,4817.14,21024
5,20251222,KRX 건설,837.89,75585
6,20251219,KRX 자동차,2515.84,12806
7,20251219,KRX 헬스케어,4861.55,30168
8,20251219,KRX 건설,820.60,36010
9,20251218,KRX 자동차,2459.08,9597


In [13]:
# 데이터프레임 csv 저장(utf-8)
df.to_csv('data/주식한달데이터.csv', index=False)